In [1]:
import torch


import processing

import get_data

import matplotlib.pyplot as plt

import numpy as np

import os

In [4]:
test_scene = 3 # can be 1,2,3
# read in the data
data_folder   = "Data/Set"+str(test_scene)

LF            = get_data.read_lightfield(data_folder)
param_dict    = get_data.read_parameters(data_folder)
depth_map     = get_data.read_depth(data_folder, highres=False)
depth_all_map = get_data.read_all_depths(data_folder, highres=False)



# convert the data to torch tensors

images     = torch.from_numpy(LF).permute([0,1,4,2,3]).float()/255.
depth_maps = torch.from_numpy(depth_all_map)

In [10]:
# Zusammengefasst:

# input : ein bild und zugehörige depth map, layers
image = images[3,3]
depth = depth_maps[3,3]
layers = 8

# erzeuge disp tensor 
disp_tensor = processing.create_disp_tensor(depth, layers)[0]

#nehme disp tensor und bild und baue mpi
mpi = processing.rgba(image, disp_tensor)

#das können wir rekonstruieren
reconstruct = processing.back_to_front_alphacomposite(mpi)

# gleich?
torch.allclose(image, reconstruct[0:3])

True

In [12]:
# jetzt naives rendering:
layers = 8

#images
image1 = images[3,3]
depth1 = depth_maps[3,3]

image2 = images[7,3]
depth2 = depth_maps[7,3]

#disp tensors 
disp_tensor1 = processing.create_disp_tensor(depth1, layers)[0]
disp_tensor2 = processing.create_disp_tensor(depth2, layers)[0]

#mpis 
mpi1 = processing.rgba(image1, disp_tensor1)
mpi2 = processing.rgba(image2, disp_tensor2)

#homography warp auf target pos [5,3]
#dafür brauchen wir die ganzen parameter

In [13]:
param_dict['baselineMM']=param_dict['baseline_mm']
param_dict['focalLength'] = param_dict['focal_length_mm']
param_dict['sensorWidthMM'] = param_dict['sensor_size_mm']

param_dict['psv_center_1_pose']=[3,3]
param_dict['psv_center_2_pose']=[3,7]
param_dict['target_image_pose']=[3,5]

param_dict['mpi_1_min_disp']=1/depth1.max()
param_dict['mpi_2_min_disp']=1/depth2.max()

param_dict['mpi_1_bin_size']=((1/depth1).max()-(1/depth1).min())/layers
param_dict['mpi_2_bin_size']=((1/depth2).max()-(1/depth2).min())/layers

In [17]:
# jetzt der homography warp
mpis = torch.stack([mpi1, mpi2], dim=0)
target_mpis = processing.homography(mpis, param_dict)

# jetzt das alpha compositing

rgba1 = processing.back_to_front_alphacomposite(target_mpis[0])
rgba2 = processing.back_to_front_alphacomposite(target_mpis[1])

# jetzt das rendering als superposition der beiden